# Ship Damage Odds
Should I roll the dice? This notebook provides the odds that will help Seafall players make that decision. We'll be using [combinations](https://en.wikipedia.org/wiki/Combination) to do the heavy lifting of the calculations. Dice rolls can be [framed](http://math.stackexchange.com/questions/900672/how-many-combinations-from-rolling-5-identical-dice) as [stars and bars](http://math.stackexchange.com/questions/208377/combination-with-repetitions) problems.

In [ ]:
import scipy.misc
import scipy.special
import numpy
import plotly
plotly.__version__

## A cursed throw during the prologue
I went to explore a region on the first island, my flag ship being supported by the second ship in my fleet. With the Woodsman as my advisor I would have 4 dice to roll where I needed three successes to avoid damage to my ship. I cast my dice and recoiled once they settled on 3 misses and merely 1 success. I felt this had to have been some rather unlucky outcome. Was I cursed?

In [ ]:
# FWIW, total number of unique outcomes from rolling 4, 6-sided dice.
scipy.misc.comb(6, 4, repetition=True) # scipy.misc.comb(# of faces, # of dice, repitition=True)
# permutation
scipy.special.perm(4,2)
# combination: asks, "Given 4 dice, which 2 ended up with a given face value."
scipy.misc.comb(4,2)

In [ ]:
cursed_probability = (1.0/3)**3 * (2.0/3) * scipy.misc.comb(4,1) + (1.0/3)**4
print cursed_probability

In [ ]:
def prob_miss(num_miss, total_dice):
    if total_dice < num_miss:
        p = 0.0
    else:
        p = (1.0/3)**num_miss * \
        (2.0/3)**(total_dice-num_miss) * \
        scipy.misc.comb(total_dice, total_dice - num_miss)
    return p

In [ ]:
cursed_probability = prob_miss(3,4) + prob_miss(4,4)
print cursed_probability

In [ ]:
def prob_miss_or_worse(num_miss, total_dice):
    if total_dice < num_miss:
        p = 0.0
    else:
        rng_miss = numpy.arange(num_miss, total_dice + 1)
        array_miss = [prob_miss(n, total_dice) for n in rng_miss]
        p = numpy.sum(array_miss)
    return p

In [ ]:
cursed_probability = prob_miss_or_worse(1,4)
print cursed_probability

# SDO matrix
There are 15 custom dice within Seafall, so a matrix should be large enough to assess the situation where all 15 are rolled. Since the early game does not use near this many dice, two matrices will be created: An early game and a late game matrix, 8 dice and 15 dice respectively

In [ ]:
import itertools
row8 = numpy.arange(1,9) # number of misses
col8 = numpy.arange(1,9) # number of dice
sdo8 = numpy.zeros((numpy.size(row8),numpy.size(row8)))
prob = [prob_miss_or_worse(*i) for i in itertools.product(row8, col8)]
ind = [tuple(numpy.subtract(i,1)) for i in itertools.product(row8, col8)]
for idx, val in enumerate(ind):
    sdo8[val] = prob[idx]

In [ ]:
# https://plot.ly/python/reference/#heatmap-colorscale
# https://plot.ly/python/heatmaps/

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly.graph_objs as go

x = ['A', 'B', 'C', 'D', 'E']
y = ['W', 'X', 'Y', 'Z']

#       x0    x1    x2    x3    x4
z = [[0.00, 0.00, 0.75, 0.75, 0.00],  # y0
     [0.00, 0.00, 0.75, 0.75, 0.00],  # y1
     [0.75, 0.75, 0.75, 0.75, 0.75],  # y2
     [0.00, 0.00, 0.00, 0.75, 0.00]]  # y3

annotations = []
for n, row in enumerate(sdo8):
    for m, val in enumerate(row):
        var = sdo8[n][m]
        annotations.append(
            dict(
                text=str(val),
                x=col8[m], y=row8[n],
                xref='x1', yref='y1',
                font=dict(color='white' if val > 0.5 else 'black'),
                showarrow=False)
            )

colorscale = [[0, '#3D9970'], [1, '#001f3f']]  # custom colorscale
trace = go.Heatmap(x=col8, y=row8, z=sdo8, colorscale=colorscale, showscale=False)

fig = go.Figure(data=[trace])
fig['layout'].update(
    title="Annotated Heatmap",
    annotations=annotations,
    xaxis=dict(ticks='', side='top'),
    # ticksuffix is a workaround to add a bit of padding
    yaxis=dict(ticks='', ticksuffix='  '),
    width=700,
    height=700,
    autosize=False
)
plotly.offline.iplot(fig, filename='Annotated Heatmap')

# A daring raid upon my province
The end of the 2nd game was nigh. I had 10 glory and was primed to secure a 2 glory treasure and victory on my next turn. I had the most glory, but it was a close game. Mike and Joe both had 9 glory. I had accumulated two treasures to this point and Mike eyed my vault enviously; I had a target on my back. Unfortunately, my ships were both at sea, so my home port was undefended. Mike took advantage and attacked! He had 6 dice to roll and needed 5 successes to raid my treasure room. I had foreseen this possibility and despite the threat I still liked my chances. Here is how the numbers break down: